**Assignment 1 Task- A**

In [ ]:
import os
import tarfile
import requests

# Define dataset URL
dataset_url = "https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz"

# Define dataset path
dataset_path = "./UrbanSound8K"

# Download the dataset
tar_file = "UrbanSound8K.tar.gz"

if not os.path.exists(tar_file):
    print("Downloading UrbanSound8K dataset...")
    response = requests.get(dataset_url, stream=True)
    with open(tar_file, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    print("Download complete.")

# Extract the dataset
if not os.path.exists(dataset_path):
    print("Extracting dataset...")
    with tarfile.open(tar_file, "r:gz") as tar:
        tar.extractall(path=dataset_path)  # Extract to target directory
    print("Extraction complete.")
else:
    print("Dataset already extracted.")


In [ ]:
!tar -xvf /content/UrbanSound8K.tar.gz -C /content/


In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

# Define window functions
def apply_windowing(y, window_type, n_fft=2048):
    if window_type == "hann":
        window = np.hanning(n_fft)
    elif window_type == "hamming":
        window = np.hamming(n_fft)
    else:  # Rectangular
        window = np.ones(n_fft)

    return window

# Load an example audio file
file_path = "./UrbanSound8K/audio/fold1/103074-7-3-2.wav"
y, sr = librosa.load(file_path, sr=None)

# Apply different windowing techniques
windows = ["hann", "hamming", "rectangular"]
n_fft = 2048

plt.figure(figsize=(12, 4))
for i, win_type in enumerate(windows):
    window = apply_windowing(y, win_type, n_fft)
    plt.subplot(1, 3, i + 1)
    plt.plot(window)
    plt.title(f"{win_type.capitalize()} Window")
plt.show()


In [10]:
def extract_features(y, sr, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfccs.T, axis=0)  # Compute mean MFCCs for classification

# Extract features for each windowing type
features = {win: extract_features(y, sr) for win in windows}


In [ ]:
import os
import tarfile
import requests
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from collections import Counter

# ------------------------------------------ #
# Step 1: Download & Extract UrbanSound8K Dataset
# ------------------------------------------ #
dataset_url = "https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz"
dataset_path = "./UrbanSound8K"
tar_file = "UrbanSound8K.tar.gz"

# Download dataset if not already downloaded
if not os.path.exists(tar_file):
    print("Downloading UrbanSound8K dataset...")
    response = requests.get(dataset_url, stream=True)
    with open(tar_file, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    print("Download complete.")

# Extract dataset if not already extracted
if not os.path.exists(dataset_path):
    print("Extracting dataset...")
    with tarfile.open(tar_file, "r:gz") as tar:
        tar.extractall(path=dataset_path)
    print("Extraction complete.")
else:
    print("Dataset already extracted.")

# ------------------------------------------ #
# Step 2: Define Windowing Techniques
# ------------------------------------------ #
def apply_windowing(n_fft, window_type):
    """Applies the selected windowing technique."""
    if window_type == "hann":
        return np.hanning(n_fft)
    elif window_type == "hamming":
        return np.hamming(n_fft)
    else:  # Rectangular Window
        return np.ones(n_fft)

# ------------------------------------------ #
# Step 3: Generate STFT Spectrograms
# ------------------------------------------ #
def plot_spectrogram(y, sr, window_type, n_fft=2048, hop_length=512):
    """Generates and plots spectrogram using STFT."""
    window = apply_windowing(n_fft, window_type)

    # Compute STFT spectrogram
    spectrogram = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    spectrogram_db = librosa.amplitude_to_db(np.abs(spectrogram))

    # Plot and save the spectrogram
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(spectrogram_db, sr=sr, hop_length=hop_length, cmap="magma")
    plt.title(f"Spectrogram ({window_type.capitalize()} Window)")
    plt.colorbar(format="%+2.0f dB")
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.show()

# ------------------------------------------ #
# Step 4: Extract MFCC Features
# ------------------------------------------ #
def extract_features(audio_path, window_type="hann", n_mfcc=13, n_fft=2048, hop_length=512):
    """Extracts MFCC features from audio using different windowing techniques."""
    y, sr = librosa.load(audio_path, sr=None)

    # Apply selected windowing technique
    window = apply_windowing(n_fft, window_type)

    # Generate spectrogram
    plot_spectrogram(y, sr, window_type, n_fft, hop_length)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    return np.mean(mfccs.T, axis=0)  # Return mean MFCCs as feature vector

# ------------------------------------------ #
# Step 5: Extract Features from UrbanSound8K
# ------------------------------------------ #
audio_folder = os.path.join(dataset_path, "UrbanSound8K/audio/fold1")  # Using fold1 for training

# Use first 300 samples for better training
X = []
Y = []
window_types = ["hann", "hamming", "rectangular"]

for window in window_types:
    print(f"Extracting features using {window.capitalize()} window...")
    for i, filename in enumerate(os.listdir(audio_folder)):
        if filename.endswith(".wav") and i < 300:  # Increased data size
            file_path = os.path.join(audio_folder, filename)
            features = extract_features(file_path, window_type=window)
            X.append(features)
            Y.append(window)  # Assign window type as label

# Convert to NumPy array
X = np.array(X)
Y = np.array(Y)

# Print feature shape
print("Feature Shape:", X.shape)
print("Label Shape:", Y.shape)

# ------------------------------------------ #
# Step 6: Train an SVM Classifier
# ------------------------------------------ #
# Encode labels
le = LabelEncoder()
Y = le.fit_transform(Y)

# Split dataset into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM classifier
svm = SVC(kernel="poly", degree=3, C=10, gamma="scale")  # Changed kernel to polynomial
svm.fit(X_train, Y_train)

# ------------------------------------------ #
# Step 7: Evaluate Model Performance
# ------------------------------------------ #
predictions = svm.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)

print(f"SVM Accuracy: {accuracy * 100:.2f}%")

# Print Label Distribution
print("Label Distribution:", Counter(Y))
